In [2]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import *
import scipy.integrate as integrate
import pandas as pd
import scipy.constants as sc
from tqdm import tqdm
import scqubits as scq

import scipy.signal as ss

In [72]:
def hamiltonian(t, args):

    if len(args) == 8:
        flux = EJ_double_pulse(t, args["P0"], args["R0"], args["T1"], args["T2"])
    else:
        flux = 1

    transmon = scq.TunableTransmon(args["Ej"], args["Ec"],0.0, flux , args["ng"], args["N"])
    
    return Qobj(transmon.hamiltonian())


def EJ_tanh_pulse(t, P0, R0, T):
    return (1-P0*(1+np.tanh(-R0+(t/T)*2*R0))/2.0)

def EJ_double_pulse(t, P0, R0, T1, T2):
    return EJ_tanh_pulse(t, P0, R0, T1) + (EJ_tanh_pulse(t - T2, -P0, R0, T1) - 1)

def Hc(Ec, Ej, N, ng):
    """
    Reutrn the charging term of the charge qubit hamiltonian as a Qobj instance
    """
    m = np.diag(4 * Ec * (np.arange(-N,N+1)-ng)**2) 
    
    return Qobj(m)

def Hj(Ec, Ej, N, ng):
    """
    Reutrn the Josephson term of the charge qubit hamiltonian as a Qobj instance
    """
    m = 0.5 * Ej * (np.diag(-np.ones(2*N), 1) +  np.diag(-np.ones(2*N), -1))
    return Qobj(m)

In [73]:
# Test Tunable Transmon

Ej= 70.0
Ec= 1.0
ng= 0.5
N = 10
P0= 0.5
R0=  100
T1= 100
T2= 350
Tmax= 300
t = np.linspace(0,T1, Tmax)

In [15]:
zp_yaml = """# zero-pi
branches:
- [JJ,1,2,70,1]
- [C,1,2,300]
- [C,2,0,200]
"""

zero_pi = scq.Circuit(zp_yaml, from_file=False)
zero_pi.sym_hamiltonian()

<IPython.core.display.Latex object>